##![Webserver Logo](https://www.designfreelogoonline.com/wp-content/uploads/2016/07/000749-online-store-logos-design-free-online-E-commerce-cart-logo-maker-02.png)

Analysis done By: N.S.Karthik

#PROJECT BRIEF:
In recent years, due to the rapid development of retail market over Internet Technologies (ecommerce), the experience of shopping has changed dramatically. As weblog data increased, the retailers started facing problem with the analysis of web data logs generated by the retail sites. We execute the analysis of  web data logs with the help of Apache Spark using data bricks community edition.


## 

sample data from weblogs :

1558143813}Daily}www.jabong.com}www.qq.com}www.jabong.com/Mens Footware/}9ab10bc2-a28b-431d-9e4f-f3dad87f3712}2}Mens Footware}1558154249742}10}true}false}NO}YES}NO}false}NO}true}3G}CDMA}WiMax}35}12}3}4}VISA}1558155246}fc_prod_view=Mens Footware}23}1280 x 720}4-bit color}NO}www.qq.comMens Footware}YES}Adobe Flash Professional CS5.5 (11.5)}Internet Explorer}192.168.10.1}iOS 8}www.jabong.com/Mens Footware}www.qq.com/5116}YES}


## We will perform following analysis on E-Commerce weblogs:

1. Session Report

* The number of sessions of time within the selected time frame 

2. Page Views Report
* The number of page views of time within the selected time frame.

3.New Visitor Report
* New Visitors are the number of distinct New users that have visited the Website during a given time period.

4.Referring Domains Report
* Referring domains are Web sites that end users visited before going to your Web site. They can indicate popular link to your Website.
* We will find number of sessions, orders received via those domain and revenue generated through that domain.

5.Target Domains Report
* Target domains are Web sites that end users are buying your products.
* We will find number of sessions, orders received via those domain and revenue generated through that domain.

6.Referring URL Report
* This report is similar to the Referring Domains Report. It displays most popular reffering URL's
* We will find number of sessions, orders received via those domain and revenue generated through that domain.

7.Revenue via IP Addresses 
* This report generates revenue via the IP addresses of visitors accessing your website in terms of number of sessions
* We will find number of sessisions, orders received via RemoteIP and revenue generated through that RemoteIP.

8.Search Query Report
* Search queries are the key words entered into Internet search engines that provided results directing end users to your Web site. 
* This report depicts the top search queries that led users to your site and allows you to compare the number of page hits received by each search query

' 9. Type of Cellular Network Technology Connection speed 
* To find what kind of mobile Connection speed used mostly by users to visit websites

10.Mobile Connection Type
* To find what kind of mobile Connection Type used mostly by users to visit websites

11.Payment Type
* To find what kind of credit/debit card used mostly by users for payment

'12. Device Screen Resolution
* To find what type of Screen Resolution users are using to buy product.

13.Browser Used for Shopping
* To find what kind of Browser used for shopping.

14.Device Type
* Type of operating system and its version used by user for shopping.

## Loading weblogs data
* From sample data we know delimiter is '}'

In [3]:
weblogs = spark.read.csv("dbfs:///FileStore/tables/Data.txt",sep= '}',schema ="Timestamp string, ReportType string, Target string, Referrer string, Link string, SessionId string, SessionCount int, PageTitle string, LoadTime string, ViewTime string, Embedded string, Cookie string, HSResponseTime string, PrefetchElement string, ElementsinHints string, HintAlreadySeen string, Viewedfor1sttimePrefetched string, Viewed1sttimenotPrefetched string, ConxSpeed string, ConxType string, PrevConxType string, VisitstoOrder string, DaystoOrder string, VisitFreq string, PurchaseFreq string, VisChip string, TimeinSession string, PreprocRules string, Secondssincelastpage string, ScreenResolution string, ColorDepth string, CookiesEnabled string, ReferringURL string, Product1stVisit string, FlashVersion string, UserAgent string, RemoteIP string, Serial string, TargetMatches string, NormalizedTarget string, ThirdPartyCookieEnabled string")

#weblogs.show(10,False)

display(weblogs)

In [4]:
weblogs.count()

In [5]:
weblogs.printSchema()

###1. Session Report

* The number of sessions of time within the selected time frame 

The session of activity that a user with a unique IP address spends on a web site during a specified period of time. The numner of user sessions on a site is used in measuring the amount of traffic a web site gets.

In [7]:
from pyspark.sql.functions import from_unixtime,sum,col

Total_SessionCount = weblogs.select(sum('SessionCount').alias('Total_SessionCount').cast('double')).rdd.map(list).collect()[0][0]

sessionCount_peruser = weblogs.groupBy(from_unixtime('Timestamp','yyyy-MM-dd').alias('Date'), from_unixtime('Timestamp','HH').alias('Hour')).agg(sum('SessionCount').alias('SessionCountByHour')).withColumn('Percentage',(col('SessionCountByHour').cast('double')/Total_SessionCount)*100.00).sort('Hour')

display(sessionCount_peruser)


### 2. Page Views Report
* The number of page views of time within the selected time frame.

In [9]:
total_PageTitles = weblogs.count()

page_Views = weblogs.groupBy(from_unixtime('Timestamp','yyyy-MM-dd').alias('Date'), from_unixtime('Timestamp','HH').alias('Hour')).agg(count('PageTitle').alias('PageViews'),(100*count('PageTitle')/total_PageTitles).alias('Percentage')).sort('Hour')

display(page_Views)

### 3.New Visitor Report
* New Visitors are the number of distinct New users that have visited the Website during a given time period.

In [11]:
NewVisitor_countByTime = weblogs.filter("Viewedfor1sttimePrefetched = 'YES'").groupBy(from_unixtime('Timestamp','yyyy-MM-dd').alias('Date'), from_unixtime('Timestamp','HH').alias('Hour')).agg(count('Viewedfor1sttimePrefetched').alias('Total_NewVisitors')).sort('Hour')

display(NewVisitor_countByTime)

### 4.Referring Domains Report
* Referring domains are Web sites that end users visited before going to your Web site. They can indicate popular link to your Website.
* We will find number of sessions, orders received via those domain and revenue generated through that domain.
* Note NormalizedTarget has referrer URL/revenue

In [13]:
from pyspark.sql.functions import split

Revenue_via_domains = weblogs.groupBy('Referrer').agg(sum('SessionCount').alias('Total_SessionCount'),count('Referrer').alias('NumberOfOrders'),sum((split('NormalizedTarget','/')[1]).cast('int')).alias('Revenue'))

display(Revenue_via_domains)

### 5.Target Domains Report
* Target domains are Web sites that end users are buying your products.
* We will find number of sessions, orders received via those domain and revenue generated through that domain.

In [15]:
TargetUrl_revenue = weblogs.groupBy('Target').agg(sum('SessionCount').alias('Total_SessionCount'),count('Target').alias('NumberOfOrders'),sum((split('NormalizedTarget','/')[1]).cast('int')).alias('Revenue'))

display(TargetUrl_revenue)

### 6.Referring URL Report
* This report is similar to the Referring Domains Report. It displays most popular reffering URL's
* We will find number of sessions, orders received via those domain and revenue generated through that domain.

In [17]:
Revenue_via_ReferringURL = weblogs.groupBy('ReferringURL').agg(sum('SessionCount').alias('Total_SessionCount'),count('ReferringURL').alias('NumberOfOrders'),sum((split('NormalizedTarget','/')[1]).cast('int')).alias('Revenue'))

display(Revenue_via_ReferringURL)

### 7.Revenue via IP Addresses 
* This report generates revenue via the IP addresses of visitors accessing your website in terms of number of sessions
* We will find number of sessisions, orders received via RemoteIP and revenue generated through that RemoteIP.

In [19]:
Revenue_via_RemoteIP = weblogs.groupBy('RemoteIP').agg(sum('SessionCount').alias('Total_SessionCount'),count('RemoteIP').alias('NumberOfOrders'),sum((split('NormalizedTarget','/')[1]).cast('int')).alias('Revenue'))

display(Revenue_via_RemoteIP)

### 8.Search Query Report
* Search queries are the key words entered into Internet search engines that provided results directing end users to your Web site. 
* This report depicts the top search queries that led users to your site and allows you to compare the number of page hits received by each search query

In [21]:
from pyspark.sql.functions import *

search_KeyWords = weblogs.groupBy((split('PreprocRules','=')[1]).alias('SearchKeyWord')).agg(count('PreprocRules').alias('Total_KeyWordCount')).sort(desc('Total_KeyWordCount'))

display(search_KeyWords)

### 9. Cellular Technology Connection speed Type
* To find what kind of mobile Connection speed used mostly by users to visit websites

In [23]:
NetworkSpeedType = weblogs.groupBy(weblogs.ConxSpeed.alias('Cellular Network Technology')).count()

display(NetworkSpeedType)

###10.Mobile Connection Type
* To find what kind of mobile Connection Type used mostly by users to visit websites

In [25]:
NetworkType = weblogs.groupby('ConxType').count()

display(NetworkType)

###11.Payment Type
* To find what kind of credit/debit card used mostly by users for payment.

In [27]:
CardType = weblogs.groupby(weblogs.VisChip.alias('CardType')).count()

display(CardType)

### 12. Device Screen Resolution
* To find what type of Screen Resolution users are using to buy product.

In [29]:
DeviceScreenResolution = weblogs.groupby('ScreenResolution').count()

display(DeviceScreenResolution)

### 13.Browser Used for Shopping
* To find what kind of Browser used for shopping.

In [31]:
Browser_used = weblogs.groupby('UserAgent').count()

display(Browser_used)

###14.Device Type
* Type of operating system and its version used by user for shopping.

In [33]:
Mobile_os = weblogs.groupby(weblogs.Serial.alias('Mobile_Operating_system')).count()

display(Mobile_os)